In [11]:
import numpy as np
import cv2
import os,sys
 
from detect import detect_pent,detect_Rnet

RorO = 48 #24 Rnet,这里修改48可以生成 Onet数据集
 
#这里修改48可以生成 Onet数据集
stdsize = RorO

pos_save_dir = "data/" + str(stdsize) + "/positive"
part_save_dir = "data/" + str(stdsize) + "/part"
neg_save_dir = "data/" + str(stdsize) + '/negative'
#这里修改48可以生成 Onet数据集
save_dir = "data/" + str(RorO)

In [12]:
def IoU(box, boxes):
    """Compute IoU between detect box and gt boxes
 
    Parameters:
    -----------
    box: numpy array , shape (5, ): x1, y1, x2, y2, score
        input box
    boxes: numpy array, shape (n, 4): x1, y1, x2, y2
        input ground truth boxes
 
    Returns:
    --------
    ovr: numpy.array, shape (n, )
        IoU
    """
    # box = (x1, y1, x2, y2)
    box_area = (box[2] - box[0] + 1) * (box[3] - box[1] + 1)
    area = (boxes[:, 2] - boxes[:, 0] + 1) * (boxes[:, 3] - boxes[:, 1] + 1)
 
    # abtain the offset of the interception of union between crop_box and gt_box
    xx1 = np.maximum(box[0], boxes[:, 0])
    yy1 = np.maximum(box[1], boxes[:, 1])
    xx2 = np.minimum(box[2], boxes[:, 2])
    yy2 = np.minimum(box[3], boxes[:, 3])
 
    # compute the width and height of the bounding box
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)
 
    inter = w * h
    ovr = inter / (box_area + area - inter)
    return ovr

In [13]:
def mkr(dr):
    if not os.path.exists(dr):
        os.mkdir(dr)

In [14]:
def read_annotation(base_dir, label_path):
    '''读取文件的image，box'''
    data = dict()
    images = []
    bboxes = []
    labelfile = open(label_path, 'r')
    while True:
        # 图像地址
        imagepath = labelfile.readline().strip('\n')
        if not imagepath:
            break
        imagepath = base_dir + '/images/' + imagepath
        images.append(imagepath)
        # 人脸数目
        nums = labelfile.readline().strip('\n')
     
        one_image_bboxes = []
        for i in range(int(nums)):
           
            bb_info = labelfile.readline().strip('\n').split(' ')
            #人脸框
            face_box = [float(bb_info[i]) for i in range(4)]
            
            xmin = face_box[0]
            ymin = face_box[1]
            xmax = xmin + face_box[2]
            ymax = ymin + face_box[3]
            
            one_image_bboxes.append([xmin, ymin, xmax, ymax])
           
        bboxes.append(one_image_bboxes)


    data['images'] = images
    data['bboxes'] = bboxes
    return data

In [15]:
data = read_annotation('data/WIDER_train/','data/wider_face_train_bbx_gt.txt')
annotations = data['bboxes']
imgs = data['images']
#annotations= np.load("labels8.npy")#[:100,4].astype(np.float32)
#imgs=np.load("imgs8.npy")
 
num = len(annotations)
print("%d pics in total" % num)
 
#这里修改48可以生成 Onet数据集
size_i = RorO

12880 pics in total


In [16]:
mkr(save_dir)
mkr(pos_save_dir)
mkr(part_save_dir)
mkr(neg_save_dir)

In [17]:
def convert_to_square(bbox):
    """Convert bbox to square
 
    Parameters:
    ----------
    bbox: numpy array , shape n x 5
        input bbox
 
    Returns:
    -------
    square bbox
    """
    square_bbox = bbox.copy()
 
    h = bbox[:, 3] - bbox[:, 1] + 1
    w = bbox[:, 2] - bbox[:, 0] + 1
    max_side = np.maximum(h,w)
    square_bbox[:, 0] = bbox[:, 0] + w*0.5 - max_side*0.5
    square_bbox[:, 1] = bbox[:, 1] + h*0.5 - max_side*0.5
    square_bbox[:, 2] = square_bbox[:, 0] + max_side - 1
    square_bbox[:, 3] = square_bbox[:, 1] + max_side - 1
    return square_bbox

In [20]:
p_idx = 0 # positive
n_idx = 0 # negative
d_idx = 0 # dont care
idx = 0
box_idx = 0

# 生成一系列txt文档用于存储Positive，Negative，Part三类数据的信息
f1 = open(os.path.join(save_dir, 'pos_' + str(stdsize) + '.txt'), 'w')
f2 = open(os.path.join(save_dir, 'neg_' + str(stdsize) + '.txt'), 'w')
f3 = open(os.path.join(save_dir, 'part_' + str(stdsize) + '.txt'), 'w')

#len(annotations) 只需要100张图片
for i in range(100):
    '''
    print(annotations[i])
    boxes = annotations[i][0:8].reshape(-1, 2)
    ix=boxes[:,0].min()
    iy=boxes[:,1].min()
    ax=boxes[:,0].max()
    ay=boxes[:,1].max()
    boxes=np.array([[ix,iy,ax,ay]])
    '''
    
    #my code
    boxes = np.array(annotations[i])
    
    #image = imgs[i].copy()
    
    #my code
    image = cv2.imread(imgs[i]).copy()
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
    if size_i == 24:
        bbos, pre_bboxes= detect_pent(image)
    else:
        pen_bbos, pen_pre_bboxes = detect_pent(image) #Pnet
        bbos, pre_bboxes = detect_Rnet(image,pen_pre_bboxes) #Rnet
 
    print("盒子数",len(pre_bboxes))
 
    if len(pre_bboxes) == 0:
        continue
 
 
    pre_bboxes = np.array(pre_bboxes)
    dets = convert_to_square(pre_bboxes)
 
    dets[:, 0:4] = np.round(dets[:, 0:4])
 
    
    #img = imgs[i]
    
    #my code
    img = cv2.imread(imgs[i])

    
    idx += 1
 
    height,width,channel = img.shape
 
    neg_num = 0
    for box in dets:
 
        x_left,y_top,x_right,y_bottom = box[0:4].astype(int)
        width = x_right - x_left + 1
        height = y_bottom - y_top + 1
        if width < 20 or x_left < 0 or y_top < 0 or x_right > img.shape[1] - 1 or y_bottom > img.shape[0] - 1:
            continue
 
        Iou = IoU(box,boxes)
 
        cropped_im = img[y_top:y_bottom + 1, x_left:x_right + 1, :]
 
        resized_im = cv2.resize(cropped_im, (stdsize, stdsize),
                                interpolation=cv2.INTER_LINEAR)
 
        # if np.max(Iou) < 0.2 and n_idx < 3.0 * p_idx + 1:
        if np.max(Iou) < 0.3 and neg_num < 60:
            save_file = os.path.join(neg_save_dir,"%s.jpg"%n_idx)
            f2.write(str(stdsize)+"/negative/%s.jpg"% n_idx + " 0\n")
            cv2.imwrite(save_file,resized_im)
            n_idx += 1
            neg_num += 1
 
        else:
            idx_Iou = np.argmax(Iou)
            assigned_gt = boxes[idx_Iou]
            x1,y1,x2,y2 = assigned_gt
 
            offset_x1 = (x1 - x_left) / float(width)
            offset_y1 = (y1 - y_top) / float(height)
            offset_x2 = (x2 - x_right) / float(width)
            offset_y2 = (y2 - y_bottom) / float(height)
            if np.max(Iou) >= 0.65:
                save_file = os.path.join(pos_save_dir, "%s.jpg" % p_idx)
                f1.write(str(stdsize)+"/positive/%s.jpg"%p_idx + ' 1 %.2f %.2f %.2f %.2f\n' % (
                    offset_x1, offset_y1, offset_x2, offset_y2))
                cv2.imwrite(save_file, resized_im)
                p_idx += 1
 
            elif np.max(Iou) >= 0.4 and d_idx < 1.0 * p_idx + 1:
                save_file = os.path.join(part_save_dir, "%s.jpg" % d_idx)
                f3.write(str(stdsize)+"/part/%s.jpg"%d_idx + ' -1 %.2f %.2f %.2f %.2f\n' % (
                    offset_x1, offset_y1, offset_x2, offset_y2))
                cv2.imwrite(save_file, resized_im)
                d_idx += 1
 
    print("%s images done, pos: %s part: %s neg: %s" % (idx, p_idx, d_idx, n_idx))
 
 
f1.close()
f2.close()
f3.close()

successful load
盒子数 1298
1 images done, pos: 0 part: 0 neg: 60
successful load
盒子数 1970
2 images done, pos: 0 part: 0 neg: 120
successful load
盒子数 688
3 images done, pos: 6 part: 7 neg: 180
successful load
盒子数 1109
4 images done, pos: 7 part: 7 neg: 240
successful load
盒子数 1987
5 images done, pos: 15 part: 14 neg: 300
successful load
盒子数 2050
6 images done, pos: 16 part: 17 neg: 360
successful load
盒子数 1822
7 images done, pos: 31 part: 32 neg: 420
successful load
盒子数 1496
8 images done, pos: 67 part: 61 neg: 480
successful load
盒子数 1589
9 images done, pos: 74 part: 75 neg: 540
successful load
盒子数 2087
10 images done, pos: 93 part: 87 neg: 600
successful load
盒子数 1115
11 images done, pos: 93 part: 91 neg: 660
successful load
盒子数 884
12 images done, pos: 99 part: 98 neg: 720
successful load
盒子数 1171
13 images done, pos: 123 part: 124 neg: 780
successful load
盒子数 783
14 images done, pos: 123 part: 124 neg: 840
successful load
盒子数 1271
15 images done, pos: 129 part: 130 neg: 900
successful

successful load
盒子数 853
99 images done, pos: 837 part: 838 neg: 5940
successful load
盒子数 2048
100 images done, pos: 838 part: 839 neg: 6000


In [21]:
save_dir = "data/48"
size = 48
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
f1 = open(os.path.join(save_dir, 'pos_%s.txt'%(size)), 'r')
f2 = open(os.path.join(save_dir, 'neg_%s.txt'%(size)), 'r')
f3 = open(os.path.join(save_dir, 'part_%s.txt'%(size)), 'r')
 
pos = f1.readlines()
neg = f2.readlines()
part = f3.readlines()
f = open(os.path.join(save_dir, 'label-train%s.txt'%(size)), 'w')
 
for i in range(int(len(pos))):
    p = pos[i].find(" ") + 1
    pos[i] = pos[i][:p-1] + " " + pos[i][p:-1] + "\n"
    f.write(pos[i])
    
for i in range(int(len(neg))):
    p = neg[i].find(" ") + 1
    neg[i] = neg[i][:p-1] + " " + neg[i][p:-1] + " -1 -1 -1 -1\n"
    f.write(neg[i])
    
for i in range(int(len(part))):
    p = part[i].find(" ") + 1
    part[i] = part[i][:p-1] + " " + part[i][p:-1] + "\n"
    f.write(part[i])

f1.close()
f2.close()
f3.close()
